In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import gradio as gr

TRAIN_PATH = "traffic_Data/DATA"
TEST_PATH = "traffic_Data/TEST"
LABELS_CSV = "labels.csv"

IMG_SIZE = (128, 128)   # HOG works best with larger images



In [2]:
labels_df = pd.read_csv(LABELS_CSV)
class_names = labels_df.set_index("ClassId")["Name"].to_dict()
class_names

{0: 'Speed limit (5km/h)',
 1: 'Speed limit (15km/h)',
 2: 'Speed limit (30km/h)',
 3: 'Speed limit (40km/h)',
 4: 'Speed limit (50km/h)',
 5: 'Speed limit (60km/h)',
 6: 'Speed limit (70km/h)',
 7: 'speed limit (80km/h)',
 8: 'Dont Go straight or left',
 9: 'Dont Go straight or Right',
 10: 'Dont Go straight',
 11: 'Dont Go Left',
 12: 'Dont Go Left or Right',
 13: 'Dont Go Right',
 14: 'Dont overtake from Left',
 15: 'No Uturn',
 16: 'No Car',
 17: 'No horn',
 18: 'Speed limit (40km/h)',
 19: 'Speed limit (50km/h)',
 20: 'Go straight or right',
 21: 'Go straight',
 22: 'Go Left',
 23: 'Go Left or right',
 24: 'Go Right',
 25: 'keep Left',
 26: 'keep Right',
 27: 'Roundabout mandatory',
 28: 'watch out for cars',
 29: 'Horn',
 30: 'Bicycles crossing',
 31: 'Uturn',
 32: 'Road Divider',
 33: 'Traffic signals',
 34: 'Danger Ahead',
 35: 'Zebra Crossing',
 36: 'Bicycles crossing',
 37: 'Children crossing',
 38: 'Dangerous curve to the left',
 39: 'Dangerous curve to the right',
 40: 'Unk

In [3]:
def load_hog_dataset(path):
    X, y = [], []
    for class_id in sorted(os.listdir(path), key=lambda x: int(x)):
        class_folder = os.path.join(path, class_id)

        if not os.path.isdir(class_folder):
            continue

        for file in os.listdir(class_folder):
            fpath = os.path.join(class_folder, file)
            img = cv2.imread(fpath)

            if img is None:
                continue

            img = cv2.resize(img, IMG_SIZE)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Compute HOG vector
            features = hog(gray,
                           orientations=9,
                           pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2),
                           block_norm='L2-Hys')

            X.append(features)
            y.append(int(class_id))

    return np.array(X), np.array(y)

X, y = load_hog_dataset(TRAIN_PATH)
print(X.shape, y.shape)


(4170, 8100) (4170,)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [5]:
# Scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)

# Train SVM
from sklearn.svm import SVC
svm = SVC(kernel="linear", C=1.0)
svm.fit(X_train_s, y_train)

# Predict
y_pred = svm.predict(X_val_s)

print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))


✅ Validation Accuracy: 0.9976019184652278

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00        52
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        39
           6       1.00      1.00      1.00        16
           7       1.00      1.00      1.00        30
           8       1.00      1.00      1.00         2
          10       1.00      1.00      1.00        14
          11       1.00      1.00      1.00        28
          12       1.00      1.00      1.00        19
          13       1.00      1.00      1.00         7
          14       1.00      1.00      1.00        26
          15       1.00      1.00      1.00         4
          16       1.00      0.93      0.96        28
          17  

In [6]:
def predict_hog(img):
    img = cv2.resize(img, IMG_SIZE)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = hog(gray,
                   orientations=9,
                   pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2),
                   block_norm='L2-Hys')

    pred = svm.predict([features])[0]
    name = class_names.get(pred, "Unknown")
    return name, pred


In [7]:
def gradio_predict(pil_img):
    img = np.array(pil_img)[:, :, ::-1]  # RGB → BGR
    name, pred_id = predict_hog(img)
    return f"{name} (Class ID: {pred_id})"

ui = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Image(type="pil", label="Upload Traffic Sign"),
    outputs=gr.Textbox(label="Prediction"),
    title="HOG + SVM Traffic Sign Classifier",
    description="Robust to blur. Uses HOG features + Linear SVM."
)




In [8]:
# Uncomment to launch
ui.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d30b66a45536905c84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
